## MLM slots evals
* Inputs:
  * raw data: `../data/raw_cx_data.json`
  * CV splits: `../data/cv_splits_10.json`

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pickle
from pathlib import Path
from hashlib import sha256
import torch
from transformers import BertTokenizerFast, BertForMaskedLM, BertModel
from import_conart import conart
from conart.mlm_masks import batched_text

In [3]:
data_path = "../data/raw_cx_data.json"
with open(data_path, "r", encoding="UTF-8") as fin:
    data = json.load(fin)
## Check data is the same
h = sha256()
h.update(pickle.dumps(data))
data_hash = h.digest().hex()[:6]
assert data_hash == "4063b4"
len(data) # should be 11642

11642

In [4]:
## Read cv splits
with open("../data/cv_splits_10.json", "r") as fin:
    cv_splits = json.load(fin)

In [5]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')
# model = BertForMaskedLM.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('ckiplab/bert-base-chinese')

## Evaluate splits

In [27]:
train_idxs, test_idxs = cv_splits[0].values()

In [28]:
xx = data[1211]
xx

{'board': 'BabyMother',
 'text': ['再', '擠', '也', '擠', '不', '出來', '了'],
 'cnstr': ['O', 'BX', 'IX', 'IX', 'IX', 'IX', 'O'],
 'slot': ['O', 'BV', 'BC', 'BV', 'BC', 'BV', 'O'],
 'cnstr_form': ['v', '也', 'v', '不', 'X'],
 'cnstr_example': ['擠', '也', '擠', '不', '出來']}

In [117]:
batch = batched_text(data, train_idxs[:5], "vslot")
list(batch.keys())

['masked', 'text', 'mindex', 'mindex_bool']

In [118]:
batch["mindex"]

array([[135, 137,  -1,  -1,  -1,  -1],
       [ 13,  15,  -1,  -1,  -1,  -1],
       [  3,   5,  -1,  -1,  -1,  -1],
       [  0,   1,   2,   4,   5,   6],
       [  0,   2,  -1,  -1,  -1,  -1]])

In [119]:
batch["masked"][0][135:138]

['[MASK]', '一', '[MASK]']

In [120]:
X = tokenizer(batch["masked"], return_tensors="pt", is_split_into_words=True, padding=True)
Xori = tokenizer(batch["text"], return_tensors="pt", is_split_into_words=True, padding=True)
ylabels = Xori["input_ids"]
out = model(**X, labels=ylabels)

In [125]:
mindex = torch.tensor(batch["mindex"]+1)
mindex

tensor([[136, 138,   0,   0,   0,   0],
        [ 14,  16,   0,   0,   0,   0],
        [  4,   6,   0,   0,   0,   0],
        [  1,   2,   3,   5,   6,   7],
        [  1,   3,   0,   0,   0,   0]])

In [127]:
torch.where(mindex > 0,
            torch.gather(ylabels, 1, mindex),
            torch.full((5,6),0))

tensor([[6678, 6678,    0,    0,    0,    0],
        [4717, 4717,    0,    0,    0,    0],
        [4020, 4020,    0,    0,    0,    0],
        [1762,  671, 6629, 1762,  671, 6629],
        [6341, 6341,    0,    0,    0,    0]])

In [142]:
torch.tensor([[1,2,3]]).T.shape

torch.Size([3, 1])

In [147]:
torch.scatter(torch.arange(30).view(6,5), 1, torch.tensor([[0,1,2,1,4,3],[1,2,3,2,0,4]]).T, 
              torch.tensor([[-1,-1,-1,-1,-1,-1],[-1,-1,-1,-1,-1,-1]]).T)

tensor([[-1, -1,  2,  3,  4],
        [ 5, -1, -1,  8,  9],
        [10, 11, -1, -1, 14],
        [15, -1, -1, 18, 19],
        [-1, 21, 22, 23, -1],
        [25, 26, 27, -1, -1]])

In [149]:
mm = torch.arange(30).view(6,5)
mm[[0,0,1,1,2,2],[0,1,1,2,2,3]]=-1
mm

tensor([[-1, -1,  2,  3,  4],
        [ 5, -1, -1,  8,  9],
        [10, 11, -1, -1, 14],
        [15, 16, 17, 18, 19],
        [20, 21, 22, 23, 24],
        [25, 26, 27, 28, 29]])

In [47]:
probs = out.logits.softmax(dim=2)

In [49]:
probs.shape

torch.Size([5, 147, 21128])

In [73]:
index_tensor = torch.tensor(batch["mindex"]).unsqueeze(2).expand(-1, -1, 21128)
seq_probs = torch.gather(probs, 1, index_tensor)

In [ ]:
probs[0]

In [86]:
tokenizer.decode(Xori["input_ids"][1])

'[CLS] 我 都 兩 三 點 才 睡 的 通 勤 中 很 累 睡 一 睡 也 還 可 以 新 左 營 到 安 平 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [85]:
tokenizer.decode(out.logits.argmax(dim=2).squeeze()[1])

'的 我 都 兩 三 點 才 睡 的 通 勤 中 很 累, 一 點 也 還 可 以 從 左 營 到 安 平 的,, 點 我 點 在 在 在, 的 睡 的,,,,, 點 我 點 在 在 在,,,,,,,,, 我 在 在 個 在 在 累,, 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 在 在 在 在 累 的, 我 中 很 的, 的 的 的 的 的 的 從 從 從 。 到 。 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的 的'

In [46]:
batch["masked"]

[['2',
  '3',
  'K',
  '輪',
  '班',
  '軍',
  '團',
  '在',
  '歐',
  '洲',
  '事',
  '不',
  '可',
  '能',
  '發',
  '生',
  '的',
  '搞',
  '不',
  '好',
  '做',
  '到',
  '一',
  '半',
  '就',
  '當',
  '初',
  '防',
  '外',
  '來',
  '難',
  '民',
  '偷',
  '渡',
  '有',
  '局',
  '部',
  '暫',
  '停',
  '申',
  '根',
  '罷',
  '工',
  '去',
  '了',
  '但',
  '他',
  '們',
  '政',
  '治',
  '因',
  '素',
  '不',
  '想',
  '跟',
  '義',
  '大',
  '利',
  '割',
  '席',
  '怕',
  '歐',
  '盟',
  '跟',
  '申',
  '根',
  '一',
  '起',
  '解',
  '體',
  '現',
  '在',
  '醫',
  '療',
  '設',
  '備',
  '供',
  '不',
  '應',
  '求',
  '只',
  '能',
  '看',
  '天',
  '了',
  '只',
  '要',
  '申',
  '根',
  '能',
  '自',
  '由',
  '出',
  '入',
  '再',
  '多',
  '口',
  '罩',
  '也',
  '沒',
  '用',
  '輕',
  '忽',
  '啊',
  '歧',
  '視',
  '口',
  '罩',
  '文',
  '化',
  '啊',
  '你',
  '看',
  '台',
  '港',
  '星',
  '起',
  '手',
  '式',
  '都',
  '限',
  '縮',
  '疫',
  '區',
  '人',
  '士',
  '搞',
  '不',
  '好',
  '吐',
  '一',
  '口',
  '痰',
  '別',
  '人',
  '[MASK]',
  '一',
  '[MASK]',
  '明',
  '天',
  '就',
  '

In [24]:
' '.join(batch["text"][0])

'23 K 輪班 軍團 在 歐洲 事 不可能 發生 的 搞不好 做到 一半 就 當初 防 外來 難民 偷渡 有 局部 暫停 申根 罷工 去 了 但 他們 政治 因素 不 想 跟 義大利 割 席 怕 歐盟 跟 申根 一起 解體 現在 醫療 設備 供不應求 只 能 看 天 了 只要 申根 能 自由 出入 再 多 口罩 也 沒用 輕忽 啊 歧視 口罩 文化 啊 你 看台 港星 起 手 式 都 限 縮 疫區 人士 搞不好 吐 一口 痰 別人 踩 一 踩 明天 就 跟 小 感冒'